In [2]:
import json
from bs4 import BeautifulSoup as bs

post_data = json.load(open('./post-query.json'))


In [3]:
post_data.keys()

dict_keys(['items', 'has_more', 'quota_max', 'quota_remaining'])

In [4]:
accepted_answers = []
from tqdm.notebook import tqdm

post_data['items'][0]['is_answered']

False

In [5]:
for item in tqdm(post_data['items']):
    if 'accepted_answer_id' in item:
        accepted_answers.append(item)


  0%|          | 0/30 [00:00<?, ?it/s]

In [6]:
len(accepted_answers)

3

In [7]:
accepted_answers

[{'tags': ['python', 'pandas'],
  'owner': {'reputation': 1986,
   'user_id': 3952994,
   'user_type': 'registered',
   'accept_rate': 76,
   'profile_image': 'https://www.gravatar.com/avatar/c2fc6a69e19964667f321ec499bdb18c?s=128&d=identicon&r=PG&f=1',
   'display_name': 'HungUnicorn',
   'link': 'https://stackoverflow.com/users/3952994/hungunicorn'},
  'is_answered': True,
  'view_count': 92740,
  'accepted_answer_id': 33975750,
  'answer_count': 5,
  'score': 63,
  'last_activity_date': 1624325486,
  'creation_date': 1448639975,
  'last_edit_date': 1501953154,
  'question_id': 33961028,
  'content_license': 'CC BY-SA 3.0',
  'link': 'https://stackoverflow.com/questions/33961028/remove-non-numeric-rows-in-one-column-with-pandas',
  'title': 'Remove non-numeric rows in one column with pandas'},
 {'tags': ['python', 'python-3.x', 'pandas'],
  'owner': {'reputation': 355,
   'user_id': 11229812,
   'user_type': 'registered',
   'profile_image': 'https://www.gravatar.com/avatar/af43f63e1

In [10]:
from stackapi import StackAPI

SITE = StackAPI('stackoverflow')
aids = [i['accepted_answer_id'] for i in accepted_answers]
answers = SITE.fetch('answers/{ids}', ids=aids, filter='withbody')


{'backoff': 0, 'has_more': False, 'page': 1, 'quota_max': 300, 'quota_remaining': 284, 'total': 0, 'items': [{'owner': {'reputation': 26, 'user_id': 12661819, 'user_type': 'registered', 'profile_image': 'https://www.gravatar.com/avatar/b1a94e11fee59e668e6d255bb5f7f2e1?s=128&d=identicon&r=PG&f=1', 'display_name': 'Dominik Berse', 'link': 'https://stackoverflow.com/users/12661819/dominik-berse'}, 'is_accepted': True, 'score': 1, 'last_activity_date': 1624315007, 'last_edit_date': 1624315007, 'creation_date': 1624314667, 'answer_id': 68075472, 'question_id': 68075343, 'content_license': 'CC BY-SA 4.0', 'body': '<p>You might want to have a look <a href="https://stackoverflow.com/a/59604826/12661819">here</a>. You can use simple integers to address the month, so you should be able to iterate like this (not tested):</p>\n<pre><code>for month in range(1, 13):\n    df_per_month = df[df[\'Date\'].dt.month == month]\n    df_per_month.to_excel(f\'{month}.xlsx\')\n</code></pre>\n<p>Edit: Note that

In [11]:
final_answer = answers['items']
id_to_answer = {
    answer['answer_id']: answer for answer in final_answer
}

In [12]:
for idx in range(len(accepted_answers)):
    accepted_answers[idx]['answer_body'] = id_to_answer[accepted_answers[idx]['accepted_answer_id']]['body']

In [13]:
qids = [i['question_id'] for i in accepted_answers]
question_bodies = SITE.fetch('questions/{ids}', ids=qids, filter='withbody')

In [14]:
question_items = question_bodies['items']
id_to_body = {
    q['question_id']: q['body'] for q in question_items
}

In [15]:
for idx in range(len(accepted_answers)):
    accepted_answers[idx]['question_body'] = id_to_body[accepted_answers[idx]['question_id']]

### For Mining Pandas APIs

In [49]:
import requests

urls = json.load(open("pandas_apis.json"))
data_frame_url = urls["df"]

def get_api_list(_url):
    page = requests.get(_url)
    soup = bs(page.content, 'html.parser')
    codes = soup.find_all("code", {"class": "xref py py-obj docutils literal notranslate"})
    apis = []
    for code in codes:
        api_name = code.find("span").text
        if "." in api_name:
            name = api_name[api_name.index(".") + 1:]
            apis.append(name)
            pass
        else:
            apis.append(api_name)
        pass
    return apis

data_frame_apis = get_api_list(data_frame_url)
# print(data_frame_apis)

list_of_apis = []
complete_api_sets = []

for key in urls:
    api_from_url = get_api_list(urls[key])
    list_of_apis.append({
        "url_key": key,
        "url": urls[key],
        "apis": api_from_url
    })
    complete_api_sets.extend(api_from_url)
    pass


print("Dataframe APIs only")
print(data_frame_apis)
print("=" * 100)
print("Pandas APIS")
complete_api_sets = list(set(complete_api_sets))
print(complete_api_sets)
print("=" * 100)
    

Dataframe APIs only
['DataFrame', 'index', 'columns', 'dtypes', 'info', 'select_dtypes', 'values', 'axes', 'ndim', 'size', 'shape', 'memory_usage', 'empty', 'set_flags', 'astype', 'convert_dtypes', 'infer_objects', 'copy', 'bool', 'head', 'at', 'iat', 'loc', 'iloc', 'insert', '__iter__', 'items', 'iteritems', 'keys', 'iterrows', 'itertuples', 'lookup', 'pop', 'tail', 'xs', 'get', 'isin', 'where', 'mask', 'query', 'add', 'sub', 'mul', 'div', 'truediv', 'floordiv', 'mod', 'pow', 'dot', 'radd', 'rsub', 'rmul', 'rdiv', 'rtruediv', 'rfloordiv', 'rmod', 'rpow', 'lt', 'gt', 'le', 'ge', 'ne', 'eq', 'combine', 'combine_first', 'apply', 'applymap', 'pipe', 'agg', 'aggregate', 'transform', 'groupby', 'rolling', 'expanding', 'ewm', 'abs', 'all', 'any', 'clip', 'corr', 'corrwith', 'count', 'cov', 'cummax', 'cummin', 'cumprod', 'cumsum', 'describe', 'diff', 'eval', 'kurt', 'kurtosis', 'mad', 'max', 'mean', 'median', 'min', 'mode', 'pct_change', 'prod', 'product', 'quantile', 'rank', 'round', 'sem', 

In [59]:
# print(data_frame_apis)

def dataframe_api_exists(line):
    for api in data_frame_apis:
        expr = api + "[ ]*\("
#         print(expr)
        if len(re.findall(expr, line)) > 0:
            return True
        pass
    return  False


# lines = ["hello()", "concat(dsaf)", "index()", "dasfasd"]
# for l in lines:
#     print(l, dataframe_api_exists(l))

In [62]:
import re 
import keyword

keywords = keyword.kwlist

def keyword_exists(line):
    for kw in keywords:
        if (kw + " ") in line or (" " + kw) in line:
            return True
    return False


def min_word_filter(texts, min_word=5):
    filtered_texts = []
    for t in texts:
        if len(t.split()) >= min_word:
            filtered_texts.append(t)
            pass
        pass
    return filtered_texts

def extract_code(text, filters):
    soup = bs(text)
    all_code = [code.text for code in soup.find_all('code')]
    for f in filters:
        all_code = f(all_code)
    return all_code
    pass

def data_frame_exists(code):
    lines = [l.strip() for l in code.split("\n")]
    data_frame_re = "[0-9]+[, \t]+[.]*"
    matches = []
    for l in lines:
        if len(re.findall(data_frame_re, l)) > 0 \
            and not keyword_exists(l) \
            and not dataframe_api_exists(l):
            matches.append(True)
        else:
            matches.append(False)
    return any(matches)

def code_exists(code):
    return False

def print_formatted_code(cid, code):
    print('\tCode Snippet %d : ' % cid)
    lines = code.split('\n')
    lines = ['\t\t' + l for l in  lines]
    print('\n'.join(lines))
    



for a in accepted_answers:
    print('=' * 100)
    print('\n')
    title = a['title']
    code_from_body = extract_code(a['question_body'], filters=[min_word_filter])
    code_from_answer = extract_code(a['answer_body'], filters=[min_word_filter])
    print("title : ", title)
    print("Code From Question : ")
    for cid, c in enumerate(code_from_body):
        is_df = data_frame_exists(c)
        print_formatted_code(cid+1, c)
        print("\t" + ("Data Frame" if is_df else "Code")  + "\n\n\n")
    print('-' * 100)
    print("Code From Answer : ")
    for cid, c in enumerate(code_from_answer):
        is_df = data_frame_exists(c)
        print_formatted_code(cid+1, c)
        print("\t" + ("Data Frame" if is_df else "Code") + "\n\n\n")
    print('-' * 100)
    print('\n')
    print('=' * 100)
    
    
    
    



title :  Remove non-numeric rows in one column with pandas
Code From Question : 
	Code Snippet 1 : 
		id, name
		1,  A
		2,  B
		3,  C
		tt, D
		4,  E
		5,  F
		de, G
		
	Data Frame



	Code Snippet 2 : 
		id, name
		1,  A
		2,  B
		3,  C
		4,  E
		5,  F
		
	Data Frame



----------------------------------------------------------------------------------------------------
Code From Answer : 
	Code Snippet 1 : 
		import pandas as pd
		from io import StringIO
		
		data = """
		id,name
		1,A
		2,B
		3,C
		tt,D
		4,E
		5,F
		de,G
		"""
		
		df = pd.read_csv(StringIO(data))
		
		In [55]: df
		Out[55]: 
		   id name
		0   1    A
		1   2    B
		2   3    C
		3  tt    D
		4   4    E
		5   5    F
		6  de    G
		
		In [56]: df[df.id.apply(lambda x: x.isnumeric())]
		Out[56]: 
		  id name
		0  1    A
		1  2    B
		2  3    C
		4  4    E
		5  5    F
		
	Data Frame



	Code Snippet 2 : 
		In [61]: df[df.id.apply(lambda x: x.isnumeric())].set_index('id')
		Out[61]: 
		   name
		id     
		1     A
		2 